# Machine Learning - Hands on Lab - Session #1

* **Lecturer:** Jonathan DEKHTIAR
* **Date:** 2017-03-13
<br/><br/>
* **Contact:** [contact@jonathandekhtiar.eu](mailto:contact@jonathandekhtiar.eu)
* **Twitter:** [@born2data](https://twitter.com/born2data)
* **LinkedIn:** [JonathanDEKHTIAR](https://fr.linkedin.com/in/jonathandekhtiar)
* **Personal Website:** [JonathanDEKHTIAR](http://www.jonathandekhtiar.eu)
* **RSS Feed:** [FeedCrunch.io](https://www.feedcrunch.io/@dataradar/)
* **Tech. Blog:** [born2data.com](http://www.born2data.com/)
* **Github:** [DEKHTIARJonathan](https://github.com/DEKHTIARJonathan)
<br/><br/>

```
*************************************************************************
**
** 2017 March 13
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

# Let's clean the data buddy !

With that general overview out of the way, let's start cleaning the Airbnb data. 

In relation to the datasets provided for the Airbnb Kaggle competition, we will focus our cleaning efforts on two files:

* **train_users_2.csv** 
* **test_users.csv** 

For now, *sessions.csv* will be left aside.

## 1. Loading the Python libraries


In [1]:
import os

import numpy as np
import pandas as pd

## 2. Loading in the Data

The first step is to load the data from the CSV files using Python. 
To do this we will use the Pandas library and load the data from two files **train_users_2.csv** and **test_users.csv**. 

After loading, we will combine them into one dataset so that any cleaning (and later any other changes) will be done to all the data at once.

In [2]:
df_train = pd.read_csv("../input/train_users_2.csv")
df_train.sample(n=5) # Only display a few lines and not the whole dataframe

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
2186,v1rq629zke,2010-10-25,20101025144723,2010-11-22,FEMALE,63.0,basic,3,en,sem-non-brand,vast,linked,Web,Mac Desktop,Chrome,US
47686,0o5mv82lgy,2012-11-07,20121107174453,2013-04-16,-unknown-,50.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,FR
123291,5t7nxmgih5,2013-11-10,20131110101429,NaN,MALE,37.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
145169,qloft1iihq,2014-01-23,20140123171746,2014-01-24,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Windows Desktop,IE,US
196041,1uqwdhzdtn,2014-05-28,20140528142730,NaN,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,NDF


In [3]:
df_test = pd.read_csv("../input/test_users.csv")
df_test.sample(n=5) # Only display a few lines and not the whole dataframe

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
59635,ar3gbf1gdw,2014-09-26,20140926235653,NaN,FEMALE,39.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome
24992,rcz5tso5ix,2014-08-05,20140805214006,NaN,FEMALE,54.0,basic,0,en,direct,direct,linked,Web,Mac Desktop,Chrome
25914,kjhi6kvzev,2014-08-07,20140807003645,NaN,FEMALE,58.0,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-
15851,eub742koit,2014-07-24,20140724065115,NaN,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-
38585,6zhionl56o,2014-08-25,20140825154040,NaN,FEMALE,27.0,facebook,0,en,other,other,marketing,Moweb,Android Phone,Chrome Mobile


In [4]:
# Combine into one dataset
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.head(n=5) # Only display a few lines and not the whole dataframe

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,20090319043255
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,20090523174809
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,20090609231247
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,20091031060129
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,20091208061105


## 3. Cleaning the timestamp

Once the data has been loaded and combined, the first cleaning step we will undertake is fixing the format of the dates.

Fixing date format is essential in order to **simplify** date manipulation.

In [5]:
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')

df_all.head(n=5) # Only display a few lines and not the whole dataframe

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2009-06-09 23:12:47
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2009-10-31 06:01:29
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2009-12-08 06:11:05


## 4. Remove booking date field

For every test data the column "date_first_booking" is missing, so we can't rely on this one for our prediction model.

**=> We must delete the column**

In [6]:
# Remove date_first_booking column
df_all.drop('date_first_booking', axis=1, inplace=True)

## 5. Clean the Age column

There are several age values that are clearly incorrect (unreasonably high or too low). 

In this step, we replace these incorrect values with "NaN", which literally stands for *Not a Number*, but implies we do not know the age value. In other words we are changing the incorrect values into missing values. 

In [7]:
# Remove outliers function
def remove_outlier(x, min_val=15, max_val=90):
    if np.isnan(x):
        return np.nan
    elif np.logical_or(x<=min_val, x>=max_val):
        return np.nan
    else:
        return x

In [8]:
# Fixing age column
df_all['age'] = df_all['age'].apply(lambda x: remove_outlier(x, min_val=15, max_val=90))
df_all['age'].fillna(-1, inplace=True)

df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
61635,direct,direct,23.0,US,2013-02-22,untracked,Safari,Mac Desktop,FEMALE,6o9af1ing7,en,Web,0,basic,2013-02-22 01:33:43
6128,direct,direct,28.0,US,2011-07-01,linked,Chrome,Mac Desktop,MALE,hanbrqxuw6,en,Web,0,basic,2011-07-01 16:56:47
187182,direct,direct,57.0,NDF,2014-05-10,untracked,-unknown-,Android Phone,FEMALE,xqkvfoq92s,en,Android,23,facebook,2014-05-10 19:17:26
214391,sem-brand,bing,-1.0,NaN,2014-07-02,omg,IE,Windows Desktop,-unknown-,rtsgry8lk2,en,Web,0,basic,2014-07-02 16:10:22
86282,sem-non-brand,google,42.0,US,2013-06-25,omg,Safari,Mac Desktop,MALE,87njh7m7jd,en,Web,0,basic,2013-06-25 01:22:39


As mentioned earlier, there are several more complicated ways to fill in the missing values in the age column. 

We are selecting this simple method for two main reasons:

* **Clarity**: this workshop is going to be long enough without adding the complication of a complex methodology for imputing missing ages.

* **Questionable results**: in my testing during the actual competition, I did test several more complex imputation methodologies. However, none of the methods I tested actually produced a better end result than the methodology outlined above.


## 6. Converting Age Column to Integers

Values inside the age columns are **floats**, it is better to have them as **integers**

In [9]:
df_all.age = df_all.age.astype(int)
df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
63966,direct,direct,-1,NDF,2013-03-07,untracked,Firefox,Windows Desktop,-unknown-,61a7yue6wv,en,Web,0,basic,2013-03-07 06:19:01
142631,direct,direct,28,NDF,2014-01-16,untracked,Chrome,Windows Desktop,MALE,lk6bs3goc3,en,Web,0,facebook,2014-01-16 19:18:02
241539,sem-brand,google,-1,NaN,2014-08-10,omg,Safari,Mac Desktop,-unknown-,4hdym8an8u,en,Web,0,basic,2014-08-10 01:55:42
197299,direct,direct,-1,US,2014-05-30,untracked,Safari,Mac Desktop,-unknown-,a4eceaxv5t,en,Web,0,basic,2014-05-30 23:28:24
44194,direct,direct,-1,NDF,2012-10-09,untracked,Mobile Safari,iPad,MALE,p6dru8mvxi,en,Web,0,basic,2012-10-09 21:54:23


## 7. Identify and fill additional columns with missing values

In [10]:
def check_NaN_Values_in_df(df):
    # searching for NaN values is all the columns
    for col in df:
        nan_count = df[col].isnull().sum()

        if nan_count != 0:
            print (col + " => "+  str(nan_count) + " NaN Values")
            
check_NaN_Values_in_df(df_all) 

country_destination => 62096 NaN Values
first_affiliate_tracked => 6085 NaN Values


It is **absolutely normal** that <i>country_destination</i> contains NaN values, it is the output field. NaN Values are test data.

However, **we need to fix** the column <i>first_affiliate_tracked</i> 

In [11]:
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)

# We check there is no more NaN except in country_destination column
check_NaN_Values_in_df(df_all) 

df_all.sample(n=5)

country_destination => 62096 NaN Values


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
37400,sem-non-brand,google,32,NDF,2012-08-21,omg,Safari,Mac Desktop,MALE,a8nghmon4i,en,Web,0,facebook,2012-08-21 04:30:25
231154,direct,direct,40,NaN,2014-07-26,untracked,-unknown-,Android Phone,MALE,z6g7cdtjqo,en,Android,23,basic,2014-07-26 19:40:43
229775,direct,direct,31,NaN,2014-07-24,untracked,Firefox,Windows Desktop,FEMALE,50ev4m5tvr,en,Web,0,basic,2014-07-24 21:44:41
94009,direct,direct,34,US,2013-07-25,untracked,Mobile Safari,iPad,-unknown-,ry1eb52mki,en,Web,0,basic,2013-07-25 20:54:55
263110,direct,direct,28,NaN,2014-09-10,untracked,Chrome,Mac Desktop,FEMALE,uzzykvr9q9,en,Web,0,basic,2014-09-10 21:01:42


## 8. Removing Users which registred before Feb. 2013

In [12]:
df_all = df_all[df_all['date_account_created'] > '2013-02-01']
df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
147119,direct,direct,38,NDF,2014-01-29,linked,Safari,Mac Desktop,FEMALE,xk75user5i,en,Web,0,basic,2014-01-29 05:46:53
91601,direct,direct,24,NDF,2013-07-16,untracked,IE,Windows Desktop,FEMALE,eprer9hg3m,en,Web,0,facebook,2013-07-16 21:10:07
241330,direct,direct,30,NaN,2014-08-09,untracked,-unknown-,iPhone,MALE,ecomxugg6n,en,iOS,25,basic,2014-08-09 19:57:38
118570,sem-non-brand,google,-1,NDF,2013-10-22,untracked,Mobile Safari,iPad,-unknown-,zam92ulcrl,en,Web,0,basic,2013-10-22 01:49:37
206329,direct,direct,37,US,2014-06-18,linked,Safari,Mac Desktop,FEMALE,vq3cugg5b2,en,Web,0,basic,2014-06-18 01:25:25


## 9. Saving the DataFrame to csv

In [13]:
# We create the output directory if necessary
if not os.path.exists("output"):
    os.makedirs("output")
    
# We export to csv
df_all.to_csv("output/cleaned.csv", sep=',', index=False)